In [1]:
import pandas as pd
from openai import OpenAI
import os
from dotenv import load_dotenv
from tqdm import tqdm
import time

In [2]:
# Load environment variables from the .env file
load_dotenv(".env")

True

In [3]:
df = pd.read_csv("data/nailsnbeyond_reviews.csv")

In [4]:
len(df)

1183

In [5]:
df.head()

,id_review,caption,relative_date,review_date,retrieval_date,rating,username,n_review_user,place_id
0,ChdDSUhNMG9nS0VJQ0FnSUNmaHJ2MnBnRRAB,took a friend had a nice experience miss savit...,an hour ago,2024-12-28 20:07:45.553247,2024-12-28 20:07:45.553306,5.0,tarun,0,ChIJu_BEcNhnUjoR1nBoaKnbQU8
1,ChdDSUhNMG9nS0VJQ0FnSUNmaHBTQi1RRRAB,Zami did an amazing job with my nails. All the...,3 hours ago,2024-12-28 20:07:45.553465,2024-12-28 20:07:45.553504,5.0,Ragavi Prabhu,62,ChIJu_BEcNhnUjoR1nBoaKnbQU8
2,ChZDSUhNMG9nS0VJQ0FnSUNmaHZDSkx3EAE,Nail art was done so neat and quick by Savitri...,3 hours ago,2024-12-28 20:07:45.553636,2024-12-28 20:07:45.553671,5.0,Nandhini kumanan,17,ChIJu_BEcNhnUjoR1nBoaKnbQU8
3,ChdDSUhNMG9nS0VJQ0FnSUR2ajdYMnNnRRAB,NaN,3 days ago,2024-12-25 20:07:45.553848,2024-12-28 20:07:45.553949,5.0,Shreeja Katama,0,ChIJu_BEcNhnUjoR1nBoaKnbQU8
4,ChdDSUhNMG9nS0VJQ0FnSUR2ajlXanFnRRAB,So so happy with my experience. Savithri and S...,3 days ago,2024-12-25 20:07:45.554079,2024-12-28 20:07:45.554116,5.0,khushi lodha,0,ChIJu_BEcNhnUjoR1nBoaKnbQU8


In [6]:
df["place_id"].value_counts()

place_id
ChIJCbZckZlnUjoRBp3RFB6IV-E    269
ChIJWXplDnFlUjoR89bQNddfMSU    257
ChIJu_BEcNhnUjoR1nBoaKnbQU8    180
ChIJY90jRy5ZqDsRT9N4d87W9to    162
ChIJVafVAUFbUjoRl9Y7PEjQN1U    157
ChIJKWj4tF5XqDsRit2Pu1lj4qc    133
ChIJw5RiJz1ZqDsRJ8iX10Y9LTs     22
ChIJGQF0CEJmUjoRdH35zqWvIeM      2
ChIJmzVJsNpnUjoRO-VbI3aMgn0      1
Name: count, dtype: int64

In [7]:
len(df["place_id"].value_counts())

9

In [8]:
df["relative_date"].value_counts()

relative_date
8 months ago     198
3 months ago     160
7 months ago     155
2 months ago     114
6 months ago      86
5 months ago      86
4 months ago      76
3 weeks ago       69
10 months ago     63
2 weeks ago       42
9 months ago      35
11 months ago     29
5 days ago        13
3 days ago        11
6 days ago        11
2 days ago         9
4 weeks ago        7
4 days ago         6
an hour ago        4
3 hours ago        4
2 hours ago        3
4 hours ago        1
7 hours ago        1
Name: count, dtype: int64

In [9]:
df["review_date"].value_counts()

review_date
2024-12-28 20:07:45.553247    1
2024-09-28 20:18:27.656940    1
2024-07-28 20:18:31.933922    1
2024-07-28 20:18:31.933748    1
2024-07-28 20:18:31.933578    1
                             ..
2024-05-28 20:13:02.302919    1
2024-05-28 20:13:02.302747    1
2024-05-28 20:13:02.302569    1
2024-05-28 20:13:02.302377    1
2024-02-28 20:22:42.797377    1
Name: count, Length: 1183, dtype: int64

In [10]:
df['caption'].isna().sum()

131

In [11]:
df = df[df['caption'].notna()]

In [12]:
len(df)

1052

In [13]:
df.head()

,id_review,caption,relative_date,review_date,retrieval_date,rating,username,n_review_user,place_id
0,ChdDSUhNMG9nS0VJQ0FnSUNmaHJ2MnBnRRAB,took a friend had a nice experience miss savit...,an hour ago,2024-12-28 20:07:45.553247,2024-12-28 20:07:45.553306,5.0,tarun,0,ChIJu_BEcNhnUjoR1nBoaKnbQU8
1,ChdDSUhNMG9nS0VJQ0FnSUNmaHBTQi1RRRAB,Zami did an amazing job with my nails. All the...,3 hours ago,2024-12-28 20:07:45.553465,2024-12-28 20:07:45.553504,5.0,Ragavi Prabhu,62,ChIJu_BEcNhnUjoR1nBoaKnbQU8
2,ChZDSUhNMG9nS0VJQ0FnSUNmaHZDSkx3EAE,Nail art was done so neat and quick by Savitri...,3 hours ago,2024-12-28 20:07:45.553636,2024-12-28 20:07:45.553671,5.0,Nandhini kumanan,17,ChIJu_BEcNhnUjoR1nBoaKnbQU8
4,ChdDSUhNMG9nS0VJQ0FnSUR2ajlXanFnRRAB,So so happy with my experience. Savithri and S...,3 days ago,2024-12-25 20:07:45.554079,2024-12-28 20:07:45.554116,5.0,khushi lodha,0,ChIJu_BEcNhnUjoR1nBoaKnbQU8
5,ChdDSUhNMG9nS0VJQ0FnSUR2dzk2WnN3RRAB,Amazing job. Service of Savithri was excellen...,5 days ago,2024-12-23 20:07:45.554246,2024-12-28 20:07:45.554283,5.0,Leena A,0,ChIJu_BEcNhnUjoR1nBoaKnbQU8


In [14]:
reviews = df["caption"]

In [15]:
len(reviews)

1052

In [16]:
reviews

0       took a friend had a nice experience miss savit...
1       Zami did an amazing job with my nails. All the...
2       Nail art was done so neat and quick by Savitri...
4       So so happy with my experience. Savithri and S...
5       Amazing job.  Service of Savithri was excellen...
                              ...                        
1177                  Lovely service, thankyou so much ❤️
1178                                           Good work.
1179    Amazing service by Suraj. Obsessed with my nails.
1180    Very friendly staff. Neat and posh environment...
1182    I have done a  gel polish recently  here it wa...
Name: caption, Length: 1052, dtype: object

In [17]:
# Function to analyze sentiment
def analyze_sentiment(text):
    """
    Analyze the sentiment of a given text using OpenAI's GPT models.

    Args:
        text (str): The text to analyze.

    Returns:
        str: The sentiment analysis result (e.g., "Positive", "Negative", "Neutral").
    """
    client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))
    
    try:
        response = client.chat.completions.create(
            model="gpt-4",  # Replace with the model you prefer
            messages=[
                {"role": "system", "content": "You are a sentiment analysis assistant."},
                {"role": "user", "content": f"Please classify the sentiment of the following text as Positive, Negative, or Neutral:\n\n{text}"}
            ],
            temperature=0.0
        )

        sentiment = response.choices[0].message.content
        return sentiment
        
    except Exception as e:
        print(f"Error during sentiment analysis: {e}")
        return None

In [19]:
response = analyze_sentiment(reviews[1])

In [20]:
response

'Positive'

In [21]:
reviews[1]

'Zami did an amazing job with my nails. All the staff were extremely courteous and polite. Loved the service.'

In [22]:
tqdm.pandas()

In [23]:
def analyze_sentiment_batch(texts):
    """
    Analyze sentiment for a batch of texts using OpenAI API.

    Args:
        texts (list of str): List of texts to analyze.

    Returns:
        list of str: List of sentiment analysis results.
    """
    client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))
    
    try:
        # Batch prompt with numbered texts for clarity
        batch_prompt = "\n".join(
            [f"Text {i+1}: {text}" for i, text in enumerate(texts)]
        )
        messages = [
            {"role": "system", "content": "You are a sentiment analysis assistant."},
            {"role": "user", "content": f"Classify the sentiment of the following texts only as either Positive, Negative, Mixed, or Neutral:\n\n{batch_prompt}." 
                    f"If you find text in Tamil, translate first and then classify."
                    f"If you find spelling or grammatical errors, correct the spelling and/or grammar and then classify." }
        ]
        
        response = client.chat.completions.create(
            model="gpt-4",  # Use "gpt-3.5-turbo" for faster response if accuracy is acceptable
            messages=messages,
            temperature=0.0
        )
        
        # Parse response into individual results
        results = response.choices[0].message.content.split("\n")
        return [result.split(":")[1].strip() for result in results]
    
    except Exception as e:
        print(f"Error during sentiment analysis: {e}")
        return [None] * len(texts)

In [24]:
len(df)

1052

In [25]:
df = df.dropna(subset="caption")

In [26]:
len(df)

1052

In [27]:
sample_df = df

In [28]:
tqdm.pandas()

batch_size = 10

sample_df['sentiment'] = None

for i in tqdm(range(0, len(sample_df), batch_size), desc="Processing Batches"):
    batch = sample_df['caption'].iloc[i:i+batch_size]
    sentiments = analyze_sentiment_batch(batch.tolist())
    sample_df.loc[batch.index, 'sentiment'] = sentiments

Processing Batches: 100%|█████████████████████| 106/106 [05:55<00:00,  3.35s/it]


In [29]:
sample_df.head()

,id_review,caption,relative_date,review_date,retrieval_date,rating,username,n_review_user,place_id,sentiment
0,ChdDSUhNMG9nS0VJQ0FnSUNmaHJ2MnBnRRAB,took a friend had a nice experience miss savit...,an hour ago,2024-12-28 20:07:45.553247,2024-12-28 20:07:45.553306,5.0,tarun,0,ChIJu_BEcNhnUjoR1nBoaKnbQU8,Positive
1,ChdDSUhNMG9nS0VJQ0FnSUNmaHBTQi1RRRAB,Zami did an amazing job with my nails. All the...,3 hours ago,2024-12-28 20:07:45.553465,2024-12-28 20:07:45.553504,5.0,Ragavi Prabhu,62,ChIJu_BEcNhnUjoR1nBoaKnbQU8,Positive
2,ChZDSUhNMG9nS0VJQ0FnSUNmaHZDSkx3EAE,Nail art was done so neat and quick by Savitri...,3 hours ago,2024-12-28 20:07:45.553636,2024-12-28 20:07:45.553671,5.0,Nandhini kumanan,17,ChIJu_BEcNhnUjoR1nBoaKnbQU8,Positive
4,ChdDSUhNMG9nS0VJQ0FnSUR2ajlXanFnRRAB,So so happy with my experience. Savithri and S...,3 days ago,2024-12-25 20:07:45.554079,2024-12-28 20:07:45.554116,5.0,khushi lodha,0,ChIJu_BEcNhnUjoR1nBoaKnbQU8,Positive
5,ChdDSUhNMG9nS0VJQ0FnSUR2dzk2WnN3RRAB,Amazing job. Service of Savithri was excellen...,5 days ago,2024-12-23 20:07:45.554246,2024-12-28 20:07:45.554283,5.0,Leena A,0,ChIJu_BEcNhnUjoR1nBoaKnbQU8,Positive


In [31]:
sample_df.to_csv("data/nailsnbeyond_sentiments.csv")

### Read Data

In [33]:
sentiments_df = pd.read_csv("data/nailsnbeyond_sentiments.csv")

In [34]:
sentiments_df.head()

,Unnamed: 0,id_review,caption,relative_date,review_date,retrieval_date,rating,username,n_review_user,place_id,sentiment
0,0,ChdDSUhNMG9nS0VJQ0FnSUNmaHJ2MnBnRRAB,took a friend had a nice experience miss savit...,an hour ago,2024-12-28 20:07:45.553247,2024-12-28 20:07:45.553306,5.0,tarun,0,ChIJu_BEcNhnUjoR1nBoaKnbQU8,Positive
1,1,ChdDSUhNMG9nS0VJQ0FnSUNmaHBTQi1RRRAB,Zami did an amazing job with my nails. All the...,3 hours ago,2024-12-28 20:07:45.553465,2024-12-28 20:07:45.553504,5.0,Ragavi Prabhu,62,ChIJu_BEcNhnUjoR1nBoaKnbQU8,Positive
2,2,ChZDSUhNMG9nS0VJQ0FnSUNmaHZDSkx3EAE,Nail art was done so neat and quick by Savitri...,3 hours ago,2024-12-28 20:07:45.553636,2024-12-28 20:07:45.553671,5.0,Nandhini kumanan,17,ChIJu_BEcNhnUjoR1nBoaKnbQU8,Positive
3,4,ChdDSUhNMG9nS0VJQ0FnSUR2ajlXanFnRRAB,So so happy with my experience. Savithri and S...,3 days ago,2024-12-25 20:07:45.554079,2024-12-28 20:07:45.554116,5.0,khushi lodha,0,ChIJu_BEcNhnUjoR1nBoaKnbQU8,Positive
4,5,ChdDSUhNMG9nS0VJQ0FnSUR2dzk2WnN3RRAB,Amazing job. Service of Savithri was excellen...,5 days ago,2024-12-23 20:07:45.554246,2024-12-28 20:07:45.554283,5.0,Leena A,0,ChIJu_BEcNhnUjoR1nBoaKnbQU8,Positive


In [35]:
sentiments_df["sentiment"].value_counts()

sentiment
Positive    998
Negative     29
Neutral      13
Mixed        12
Name: count, dtype: int64

In [41]:
new_df = sentiments_df[~sentiments_df["sentiment"].isin(["Positive", "Neutral", "Negative", "Mixed"])]

In [42]:
len(new_df)

0

In [43]:
new_df

,Unnamed: 0,id_review,caption,relative_date,review_date,retrieval_date,rating,username,n_review_user,place_id,sentiment


In [44]:
len(df)

1052

In [45]:
new_df["caption"].isna().sum()

0